In [1]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.MiscUtils import *
from Utils.ImageUtils import *
from Utils.MathUtils import *
import scipy.optimize 
square_side = 12.5


In [2]:
def getRotationAndTrans(A, all_H):
    all_RT = []
    for H in all_H:
        h1 = H[:, 0]
        h2 = H[:, 1]
        h3 = H[:, 2]
        lamb = np.linalg.norm(np.dot(np.linalg.inv(A), h1), 2)

        r1 = np.dot(np.linalg.inv(A), h1) / lamb
        r2 = np.dot(np.linalg.inv(A), h2) / lamb
        r3 = np.cross(r1, r2)
        t = np.dot(np.linalg.inv(A), h3) / lamb
        RT = np.vstack((r1, r2, r3, t)).T
        all_RT.append(RT)

    return all_RT        

In [3]:
def extractParamFromA(init_A, init_kc):
    alpha = init_A[0,0]
    gamma = init_A[0,1]
    beta = init_A[1,1]
    u0 = init_A[0,2]
    v0 = init_A[1,2]
    k1 = init_kc[0]
    k2 = init_kc[1]

    x0 = np.array([alpha, gamma, beta, u0, v0, k1, k2])
    return x0

In [4]:
def retrieveA(x0):
    alpha, gamma, beta, u0, v0, k1, k2 = x0
    A = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]]).reshape(3,3)
    kc = np.array([k1, k2]).reshape(2,1)
    return A, kc


In [5]:
def lossFunc(x0, init_all_RT, all_image_corners, world_corners):

    A, kc = retrieveA(x0)
    alpha, gamma, beta, u0, v0, k1, k2 = x0

    error_mat = []

    for i, image_corners in enumerate(all_image_corners): # for all images

        #RT for 3d world points
        RT = init_all_RT[i]
        #get ART for 2d world points
        RT3 = np.array([RT[:,0], RT[:,1], RT[:,3]]).reshape(3,3) #review
        RT3 = RT3.T
        ART3 = np.dot(A, RT3)

        image_total_error = 0

        for j in range(world_corners.shape[0]):

            world_point_2d = world_corners[j]
            world_point_2d_homo = np.array([world_point_2d[0], world_point_2d[1], 1]).reshape(3,1)
            world_point_3d_homo = np.array([world_point_2d[0], world_point_2d[1], 0, 1]).reshape(4,1)

            #get radius of distortion
            XYZ = np.dot(RT, world_point_3d_homo)
            x =  XYZ[0] / XYZ[2]
            y = XYZ[1] / XYZ[2]
            # x = alpha * XYZ[0] / XYZ[2] #assume gamma as 0 
            # y = beta * XYZ[1] / XYZ[2] #assume gamma as 0
            r = np.sqrt(x**2 + y**2) #radius of distortion

            #observed image co-ordinates
            mij = image_corners[j]
            mij = np.array([mij[0], mij[1], 1], dtype = 'float').reshape(3,1)

            #projected image co-ordinates
            uvw = np.dot(ART3, world_point_2d_homo)
            u = uvw[0] / uvw[2]
            v = uvw[1] / uvw[2]

            u_dash = u + (u - u0) * (k1 * r**2 + k2 * r**4)
            v_dash = v + (v - v0) * (k1 * r**2 + k2 * r**4)

            mij_dash = np.array([u_dash, v_dash, 1], dtype = 'float').reshape(3,1)

            #get error
            e = np.linalg.norm((mij - mij_dash), ord=2)
            image_total_error = image_total_error + e

        error_mat.append(image_total_error)
    
    return np.array(error_mat)
        

In [6]:
folder_name = "/home/sakshi/courses/CMSC733/sakshi_hw1/Data/Calibration_Imgs"
images = loadImages(folder_name)

all_corners = getImagesHomoPoints(images)
displayCorners(images, all_corners)

print("Calculating H for %d images", len(images))
all_H = getAllH(all_corners, square_side)
print("Calculating B")
B = getB(all_H)
print("Estimated B = ", B)
print("Calculating A")
A = getA(B)
print("Estimated A = ",A)
print("Calculating rotation and translation")
all_RT = getRotationAndTrans(A, all_H)
print("Init Kc")
kc = np.array([0,0]).reshape(2,1)



Loading images from  /home/sakshi/courses/CMSC733/sakshi_hw1/Data/Calibration_Imgs
Calculating H for %d images 13
Calculating B
B matrix is  [-1.47821817e-07 -2.81891939e-10 -1.49419421e-07  1.12211956e-04
  2.03206168e-04 -9.99999973e-01]
Estimated B =  [[-1.47821817e-07 -2.81891939e-10  1.12211956e-04]
 [-2.81891939e-10 -1.49419421e-07  2.03206168e-04]
 [ 1.12211956e-04  2.03206168e-04 -9.99999973e-01]]
Calculating A
Estimated A =  [[ 2.07920199e+03 -3.94373138e+00  7.56512097e+02]
 [ 0.00000000e+00  2.06806034e+03  1.35854437e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Calculating rotation and translation
Init Kc


In [16]:
x0 = extractParamFromA(A, kc)
world_corners = getWorldPoints(square_side)
kc = np.array([1, 1]).reshape(2,1)

In [17]:
res = scipy.optimize.least_squares(fun=lossFunc, x0=x0, method="lm", args=[all_RT, all_corners, world_corners])
x1 = res.x

In [15]:
retrieveA(x1)

(array([[ 2.07920199e+03, -3.94373138e+00,  7.56512097e+02],
        [ 0.00000000e+00,  2.06806034e+03,  1.35854437e+03],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[0.],
        [0.]]))

In [10]:
retrieveA(x0)

(array([[ 2.07920199e+03, -3.94373138e+00,  7.56512097e+02],
        [ 0.00000000e+00,  2.06806034e+03,  1.35854437e+03],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[0],
        [0]]))

In [11]:
lossFunc(x0, all_RT, all_corners, world_corners)

array([36.31285821, 39.70058789, 67.7936188 , 26.48221224, 54.837745  ,
       32.94590383, 52.54583402, 33.91290422, 46.77450363, 17.44010409,
       39.85952894, 38.81430174, 37.00076977])